## 37.3 Supervised NLP

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en_core_web_sm
spacy.load('en_core_web_sm')

[nltk_data] Downloading package gutenberg to C:\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450


,imperiously,Mr,calculation,approve,torment,profound,highly,cat,altogether,currant,...,make,man,ready,toast,bat,servant,decency,buttered,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.968421052631579

Test set score: 0.7526315789473684


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8631578947368421


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.887719298245614

Test set score: 0.7894736842105263


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.452


col_0,Austen,Carroll
row_0,,
Austen,72,78
Carroll,59,41


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0

In [15]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion_base = re.sub(r'Chapter \d+', '', persuasion)
alice_base = re.sub(r'CHAPTER .*', '', alice)

In [16]:
alice = text_cleaner(alice_base)
persuasion = text_cleaner(persuasion_base)
# question: the text cleaner appears to clean punctuation but the below text still includes it...

In [17]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [18]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [19]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [20]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.

# POS dict
pos_d = {'VERB':'verb_ct', 'NOUN':'noun_ct', 'ADV':'adv_ct', 'ADP':'adp_ct', 
         'PROPN':'propn_ct', 'ADJ':'adj_ct', 'DET':'det_ct', 'PUNCT':'punct_ct'}

def bow_features(quotes, common_words, **kwargs):
    print(len(quotes))
    
    # sentence stats
    sent_stats = ['comma_ct', 'word_ct', 'adv_ct', 'adp_ct', 'propn_ct', 'adj_ct', 'punct_ct'] #'verb_ct', 'noun_ct','det_ct',
    if 'sent_stats' in kwargs:
        df = pd.DataFrame(columns=list(common_words) + sent_stats)
        cols = list(common_words)+sent_stats
    else:
        df = pd.DataFrame(columns=common_words)
        cols = list(common_words)
    
    for col in df.columns:
        df[col] = np.zeros(len(quotes[0]))
    df['text_sentence'] = quotes[0] 
    df['text_source'] = quotes[1]
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, quote in enumerate(df['text_sentence']):
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in quote
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            try:
                df.loc[i, word] += 1
            except:
                print(word)
        
        # add sentence features
        if 'sent_stats' in kwargs:
            commas = 0
            for token in quote:
                if token.orth_ == ',':
                    commas += 1
            df.loc[i, 'comma_ct'] = commas
                    
            c = Counter([token.pos_ for token in quote])
            for key in pos_d.keys():
                if key in c.keys():
                    df.loc[i, pos_d[key]] = c[key]
                else:
                    df.loc[i, pos_d[key]] = c[key]
            
            df.loc[i, 'word_ct'] = len([token for token in quote if (not token.is_punct)])

        # This counter is just to make sure the kernel didn't hang.
        if i % 1000 == 0:
            print("Processing row {}".format(i))
            
    return df

In [21]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [22]:
word_counts = bow_features(sentences, common_words)

5848
Processing row 0
Processing row 1000
Processing row 2000
Processing row 3000
Processing row 4000
Processing row 5000


In [23]:
#includes grammatical features
word_counts_ss = bow_features(sentences, common_words, sent_stats= True)

5848
Processing row 0
Processing row 1000
Processing row 2000
Processing row 3000
Processing row 4000
Processing row 5000


In [24]:
def make_train_test(word_count_df):
    Y = word_count_df['text_source']
    X = word_count_df.iloc[:, ~word_count_df.columns.isin(['text_sentence','text_source'])]

    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        Y,
                                                        test_size=0.3,
                                                        random_state=0)
    return X_train, X_test, y_train, y_test

In [25]:
X_train_ss, X_test_ss, y_train_ss, y_test_ss = make_train_test(word_counts_ss)

In [26]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8631578947368421


In [27]:
#before adding features, lets try to improve the model by fine tuning it

from sklearn.model_selection import GridSearchCV
parameters = {
            'penalty':['l2'],
            'C':[1,10,100],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
        }
lr = LogisticRegression()
GS = GridSearchCV(lr, parameters,cv=6,verbose=10)
GS.fit(X_train,y_train)

new_params = GS.best_params_

Fitting 6 folds for each of 12 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] C=1, penalty=l2, solver=newton-cg ...............................
[CV] ... C=1, penalty=l2, solver=newton-cg, score=0.771, total=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] C=1, penalty=l2, solver=newton-cg ...............................
[CV] ... C=1, penalty=l2, solver=newton-cg, score=0.875, total=   0.1s


...

C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ....... C=100, penalty=l2, solver=sag, score=0.723, total=   0.4s
[CV] C=100, penalty=l2, solver=sag ...................................


C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ....... C=100, penalty=l2, solver=sag, score=0.830, total=   0.4s


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    9.4s finished


In [28]:
print(new_params)

{'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}


In [29]:
lr = LogisticRegression(C=1.0, penalty='l2', solver='newton-cg')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8631578947368421


In [30]:
lr = LogisticRegression(C=1.0, penalty='l2', solver='newton-cg')
train = lr.fit(X_train_ss, y_train_ss)
print(X_train_ss.shape, y_train_ss.shape)
print('Training set score:', lr.score(X_train_ss, y_train_ss))
print('\nTest set score:', lr.score(X_test_ss, y_test_ss))

(4093, 3095) (4093,)
Training set score: 0.9188859027608112

Test set score: 0.8712250712250712


Adding grammatical features and fine tuning the logistic regression model seems to help bridge the overfitting problem although the overall accuracy didn't improve all that much.

In [31]:
from sklearn.svm import LinearSVC

svclin_clf = LinearSVC(random_state=13)
train = svclin_clf.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svclin_clf.score(X_train, y_train))
print('\nTest set score:', svclin_clf.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8


In [32]:
svclin_clf = LinearSVC(random_state=13)
train = svclin_clf.fit(X_train_ss, y_train_ss)
print(X_train_ss.shape, y_train_ss.shape)
print('Training set score:', svclin_clf.score(X_train_ss, y_train_ss))
print('\nTest set score:', svclin_clf.score(X_test_ss, y_test_ss))

C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(4093, 3095) (4093,)
Training set score: 0.9582213535304178

Test set score: 0.8672364672364672


## Challenge 1

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [33]:
#Load in and clean a new book, using our text_cleaner function and the spacy load function
paradise = gutenberg.raw('milton-paradise.txt')
paradise = re.sub(r'CHAPTER .*', '', paradise)
paradise = text_cleaner(paradise)

#Spacy load
paradise_doc = nlp(paradise)

In [34]:
#Extract and store sentences from spacy doc
paradise_sents = [[sent, 'Milton'] for sent in paradise_doc.sents]

#Add the paradise sentences to our existing sentences DF
sentences = pd.DataFrame(alice_sents + persuasion_sents + paradise_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [35]:
paradise_sentences = pd.DataFrame(paradise_sents)
paradise_bow = bow_features(paradise_sentences, common_words)

3290
Processing row 0
Processing row 1000
Processing row 2000
Processing row 3000


In [36]:
paradise_bow.fillna(0,inplace=True)

In [37]:
print(word_counts_ss.shape)

(5848, 3097)


In [38]:
paradise_bow2 = paradise_bow.iloc[:,0:3087]

In [39]:
word_counts_ss2 = word_counts_ss.iloc[:,0:3087]

In [40]:
print(word_counts_ss2.shape)
print(paradise_bow2.shape)

(5848, 3087)
(3290, 3087)


In [50]:
alice_wc = word_counts[word_counts.text_source == 'Carroll']

In [51]:
alice_wc.fillna(0,inplace=True)

C:\Users\kylem\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [53]:
print(alice_wc.shape)
print(paradise_bow2.shape)

(1974, 3087)
(3290, 3087)


In [54]:
# Identifying variables
X_paradise = paradise_bow2.drop(['text_sentence','text_source'], 1)
y_paradise = paradise_bow2.text_source


X_alice = alice_wc.drop(['text_sentence','text_source'], 1)
y_alice = alice_wc.text_source

# Combine the Paradise sentence data with the Alice data from the test set.
X_pa = pd.concat([X_paradise, X_alice], 0)
y_pa = pd.concat([y_paradise, y_alice], 0)

In [55]:
X_train_pa, X_test_pa, y_train_pa, y_test_pa = train_test_split(X_pa, y_pa, test_size=0.3, random_state=0)

In [56]:
lr = LogisticRegression(C=1.0, penalty='l2', solver='newton-cg')
train = lr.fit(X_train_pa, y_train_pa)
print(X_train_pa.shape, y_train_pa.shape)
print('Training set score:', lr.score(X_train_pa, y_train_pa))
print('\nTest set score:', lr.score(X_test_pa, y_test_pa))

(3684, 3085) (3684,)
Training set score: 0.9248099891422367

Test set score: 0.8734177215189873


In [57]:
lr_pa_predicted = lr.predict(X_test_pa)
pd.crosstab(y_test_pa, lr_pa_predicted)

col_0,Carroll,Milton
text_source,,
Carroll,446,149
Milton,51,934


In [58]:
persuasion_wc = word_counts[word_counts.text_source == 'Austen']
persuasion_wc.shape

(3874, 3087)

In [61]:
X_paradise = paradise_bow2.drop(['text_sentence','text_source'], 1)
y_paradise = paradise_bow2.text_source


X_persuasion = persuasion_wc.drop(['text_sentence','text_source'], 1)
y_persuasion = persuasion_wc.text_source

# Combine the Paradise sentence data with the Alice data from the test set.
X_pp = pd.concat([X_paradise, X_persuasion], 0)
y_pp = pd.concat([y_paradise, y_persuasion], 0)

# Split into train and test sets
X_train_pp, X_test_pp, y_train_pp, y_test_pp = train_test_split(X_pp, y_pp, test_size=0.4, random_state=0)

# Model.
lr = LogisticRegression()
lr.fit(X_train_pp, y_train_pp)
print('Training set score:', lr.score(X_train_pp, y_train_pp))
print('\nTest set score:', lr.score(X_test_pp, y_test_pp))

Training set score: 0.932759422987436

Test set score: 0.8715980460572226


In [62]:
lr_pp_predicted = lr.predict(X_test_pp)
pd.crosstab(y_test_pp, lr_pp_predicted)

col_0,Austen,Milton
text_source,,
Austen,1412,145
Milton,223,1086
